In [1]:
from utils import *

In [2]:
import os
import numpy as np

In [3]:
# Download amass data from 

#https://amass.is.tue.mpg.de/download.php

In [4]:
#amass_npz_fname = '../../../thirdParty/amass/support_data/github_data/amass_sample.npz' # the path to body data
amass_npz_fname = '../../../thirdParty/accad/C2 - Run to stand_poses.npz' # the path to body data
bdata = np.load(amass_npz_fname)

In [5]:
mc = MayaController()

In [6]:
# Test Maya command
mc.SetCurrentTimeFrame(10)

In [7]:
print('Data keys available:%s'%list(bdata.keys()))

Data keys available:['trans', 'gender', 'mocap_framerate', 'betas', 'dmpls', 'poses']


In [8]:
bdata['mocap_framerate']

array(120.)

In [10]:
# Get skeleton information from https://meshcapade.wiki/SMPL#smpl-x
SMPL_SKELETON = {
    0: 'Pelvis',     3: 'Spine1',       6: 'Spine2',    9: 'Spine3',    12: 'Neck',     15: 'Head',
     1: 'L_Hip',      4: 'L_Knee',       7: 'L_Ankle',  10: 'L_Foot',
     2: 'R_Hip',      5: 'R_Knee',       8: 'R_Ankle',  11: 'R_Foot',
    13: 'L_Collar',  16: 'L_Shoulder',  18: 'L_Elbow',  20: 'L_Wrist',
    14: 'R_Collar',  17: 'R_Shoulder',  19: 'R_Elbow',  21: 'R_Wrist',
    22: 'L_Hand',
    23: 'R_Hand'
}

SMPL_H_SKELETON = {
    0: 'Pelvis',     3: 'Spine1',       6: 'Spine2',    9: 'Spine3',    12: 'Neck',     15: 'Head',
    1: 'L_Hip',      4: 'L_Knee',       7: 'L_Ankle',  10: 'L_Foot',
    2: 'R_Hip',      5: 'R_Knee',       8: 'R_Ankle',  11: 'R_Foot',
    13: 'L_Collar',  16: 'L_Shoulder',  18: 'L_Elbow',  20: 'L_Wrist',
    14: 'R_Collar',  17: 'R_Shoulder',  19: 'R_Elbow',  21: 'R_Wrist',
    22: 'lindex0',  23: 'lindex1',  24: 'lindex2',
    25: 'lmiddle0', 26: 'lmiddle1', 27: 'lmiddle2',
    28: 'lpinky0',  29: 'lpinky1',  30: 'lpinky2',
    31: 'lring0',   32: 'lring1',   33: 'lring2',
    34: 'lthumb0',  35: 'lthumb1',  36: 'lthumb2',
    37: 'rindex0',  38: 'rindex1',  39: 'rindex2',
    40: 'rmiddle0', 41: 'rmiddle1', 42: 'rmiddle2',
    43: 'rpinky0',  44: 'rpinky1',  45: 'rpinky2',
    46: 'rring0',   47: 'rring1',   48: 'rring2',
    49: 'rthumb0',  50: 'rthumb1',  51: 'rthumb2'
}

In [9]:
# number of joints
len(bdata['poses'][0]) // 3

52

In [11]:
bdata['trans'].shape # frame , (x,y,z)

(639, 3)

In [ ]:
# Debug only
#root_rotation =  bdata['poses'][0][0:3]
# root_translate = -bdata['trans'][0]
# joint_root = "f_avg_root"
# mc.SetObjectLocalRotation(joint_root, root_rotation)
# mc.SetObjectWorldTransform(joint_root, root_translate)

# root_translate

# for i in range(1,22):
#     rotation = np.rad2deg(bdata['poses'][0][i*3:(i+1)*3])
#     joint_name = "f_avg_" + SMPL_SKELETON[i]
#     print(joint_name,rotation)
#     mc.SetObjectLocalRotation(joint_name, rotation)
#     #break

#  translateX,translateY,translateZ = bdata['trans'][1]

In [12]:
def SetOneFrame(frame, body_data, joints_info_list: list):
    '''
    Validation:
    Set one maya frame from load data
    :param frame: int
    :param body_data: array
    :param joints_info_list: list
    :return:
    '''
    #  set current time frame
    mc.SetCurrentTimeFrame(frame)

    for i in range(0,len(joints_info_list)):
        # get rotation from body data
        rotateX, rotateY, rotateZ = np.rad2deg(body_data['poses'][frame][i*3:(i+1)*3])
        
        # get joint name
        joint_name = "f_avg_" + joints_info_list[i]
        
        # if i == 0, set root translation
        if i == 0:
            joint_name = "f_avg_root"
            translateX,translateY,translateZ = body_data['trans'][frame]
            mc.SetObjectAttribute(joint_name, "translateX", translateX)
            mc.SetObjectAttribute(joint_name, "translateY", translateY)
            mc.SetObjectAttribute(joint_name, "translateZ", translateZ)
            
        # set rotation for joints
        mc.SetObjectAttribute(joint_name, "rotateX", rotateX)
        mc.SetObjectAttribute(joint_name, "rotateY", rotateY)
        mc.SetObjectAttribute(joint_name, "rotateZ", rotateZ)
        
        mc.SetCurrentKeyFrameForPositionAndRotation(joint_name)
        #break

In [13]:
from tqdm.auto import tqdm

In [14]:
for frame in tqdm(range(0, len(bdata["poses"]), 10)):
    SetOneFrame(frame, bdata, SMPL_H_SKELETON)

  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
mc.UndoToBeginning(10000)